In [1]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()
from nltk.stem.lancaster import LancasterStemmer
lc = LancasterStemmer()
from nltk.stem import SnowballStemmer
sb = SnowballStemmer("english")

In [3]:
from os.path import join
from pymagnitude import *
#from utils import *


def w2v_pymagnitude(train_text, path, name):
    train_corpus = [text_to_word_sequence(text) for text in train_text]
    model = Magnitude(path)
    
    result = []
    true, false = 0, 0
    oov = []
    for text in train_corpus:
        vec = np.zeros(model.dim)
        for n_w, word in enumerate(text):
            if word in model: #0.9906
                true += 1
                vec = vec + model.query(word)
                continue
            word_ = word.upper()
            if word_ in model: #0.9909
                true += 1
                vec = vec + model.query(word_)
                continue
            word_ = word.capitalize()
            if word_ in model: #0.9925
                true += 1
                vec = vec + model.query(word_)
                continue
            word_ = ps.stem(word)
            if word_ in model: #0.9927
                true += 1
                vec = vec + model.query(word_)
                continue
            word_ = lc.stem(word)
            if word_ in model: #0.9932
                true += 1
                vec = vec + model.query(word_)
                continue
            word_ = sb.stem(word)
            if word_ in model: #0.9933
                true += 1
                vec = vec + model.query(word_)
                continue
            false += 1
            vec = vec + model.query(word)
            oov.append(word)
                
        vec = vec / (n_w + 1)
        result.append(vec)
    print(true, false, true/(true+false))

    w2v_cols = ["{}_mag{}".format(name, i) for i in range(1, model.dim+1)]
    result = pd.DataFrame(result)
    result.columns = w2v_cols
    
    return result, oov

[nltk_data] Downloading package punkt to /home/keras/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Using TensorFlow backend.


In [9]:
def analyzer_bow(text):
    stop_words = ['i', 'a', 'an', 'the', 'to', 'and', 'or', 'if', 'is', 'are', 'am', 'it', 'this', 'that', 'of', 'from', 'in', 'on']
    text = text.lower() # 小文字化
    text = text.replace('\n', '') # 改行削除
    text = text.replace('\t', '') # タブ削除
    text = re.sub(re.compile(r'[!-\/:-@[-`{-~]'), ' ', text) # 記号をスペースに置き換え
    for bad_word in contraction_mapping:
        if bad_word in text:
            text = text.replace(bad_word, contraction_mapping[bad_word])
    text = text.split(' ') # スペースで区切る
    text = [sb.stem(t) for t in text]
    
    words = []
    for word in text:
        if (re.compile(r'^.*[0-9]+.*$').fullmatch(word) is not None): # 数字が含まれるものは分割
            for w in re.findall(r'(\d+|\D+)', word):
                words.append(w)
            continue
        if word in stop_words: # ストップワードに含まれるものは除外
            continue
        if len(word) < 2: #  1文字、0文字（空文字）は除外
            continue
        words.append(word)
        
    return " ".join(words)

def analyzer_embed(text):
    text = text.lower() # 小文字化
    text = text.replace('\n', '') # 改行削除
    text = text.replace('\t', '') # タブ削除
    puncts = r',.":)(-!?|;\'$&/[]>%=#*+\\•~@£·_{}©^®`<→°€™›♥←×§″′Â█½à…“★”–●â►−¢²¬░¶↑±¿▾═¦║―¥▓—‹─▒：¼⊕▼▪†■’▀¨▄♫☆é¯♦¤▲è¸¾Ã⋅‘∞∙）↓、│（»，♪╩╚³・╦╣╔╗▬❤ïØ¹≤‡√。【】'
    for punct in puncts:
        text = text.replace(punct, f' {punct} ')
    for bad_word in contraction_mapping:
        if bad_word in text:
            text = text.replace(bad_word, contraction_mapping[bad_word])
    text = text.split(' ') # スペースで区切る
    
    words = []
    for word in text:
        if (re.compile(r'^.*[0-9]+.*$').fullmatch(word) is not None): # 数字が含まれるものは分割
            for w in re.findall(r'(\d+|\D+)', word):
                words.append(w)
            continue
        if len(word) < 1: #  0文字（空文字）は除外
            continue
        words.append(word)
        
    return " ".join(words)

contraction_mapping = {u"ain’t": u"is not", u"aren’t": u"are not",u"can’t": u"cannot", u"’cause": u"because",
                       u"could’ve": u"could have", u"couldn’t": u"could not", u"didn’t": u"did not",
                       u"doesn’t": u"does not", u"don’t": u"do not", u"hadn’t": u"had not",
                       u"hasn’t": u"has not", u"haven’t": u"have not", u"he’d": u"he would",
                       u"he’ll": u"he will", u"he’s": u"he is", u"how’d": u"how did", u"how’d’y": u"how do you",
                       u"how’ll": u"how will", u"how’s": u"how is",  u"I’d": u"I would",
                       u"I’d’ve": u"I would have", u"I’ll": u"I will", u"I’ll’ve": u"I will have",
                       u"I’m": u"I am", u"I’ve": u"I have", u"i’d": u"i would", u"i’d’ve": u"i would have",
                       u"i’ll": u"i will",  u"i’ll’ve": u"i will have",u"i’m": u"i am", u"i’ve": u"i have",
                       u"isn’t": u"is not", u"it’d": u"it would", u"it’d’ve": u"it would have",
                       u"it’ll": u"it will", u"it’ll’ve": u"it will have",u"it’s": u"it is",
                       u"let’s": u"let us", u"ma’am": u"madam", u"mayn’t": u"may not",
                       u"might’ve": u"might have",u"mightn’t": u"might not",u"mightn’t’ve": u"might not have",
                       u"must’ve": u"must have", u"mustn’t": u"must not", u"mustn’t’ve": u"must not have",
                       u"needn’t": u"need not", u"needn’t’ve": u"need not have",u"o’clock": u"of the clock",
                       u"oughtn’t": u"ought not", u"oughtn’t’ve": u"ought not have", u"shan’t": u"shall not", 
                       u"sha’n’t": u"shall not", u"shan’t’ve": u"shall not have", u"she’d": u"she would",
                       u"she’d’ve": u"she would have", u"she’ll": u"she will", u"she’ll’ve": u"she will have",
                       u"she’s": u"she is", u"should’ve": u"should have", u"shouldn’t": u"should not",
                       u"shouldn’t’ve": u"should not have", u"so’ve": u"so have",u"so’s": u"so as",
                       u"this’s": u"this is",u"that’d": u"that would", u"that’d’ve": u"that would have",
                       u"that’s": u"that is", u"there’d": u"there would", u"there’d’ve": u"there would have",
                       u"there’s": u"there is", u"here’s": u"here is",u"they’d": u"they would", 
                       u"they’d’ve": u"they would have", u"they’ll": u"they will", 
                       u"they’ll’ve": u"they will have", u"they’re": u"they are", u"they’ve": u"they have", 
                       u"to’ve": u"to have", u"wasn’t": u"was not", u"we’d": u"we would",
                       u"we’d’ve": u"we would have", u"we’ll": u"we will", u"we’ll’ve": u"we will have", 
                       u"we’re": u"we are", u"we’ve": u"we have", u"weren’t": u"were not",
                       u"what’ll": u"what will", u"what’ll’ve": u"what will have", u"what’re": u"what are",
                       u"what’s": u"what is", u"what’ve": u"what have", u"when’s": u"when is",
                       u"when’ve": u"when have", u"where’d": u"where did", u"where’s": u"where is",
                       u"where’ve": u"where have", u"who’ll": u"who will", u"who’ll’ve": u"who will have",
                       u"who’s": u"who is", u"who’ve": u"who have", u"why’s": u"why is", u"why’ve": u"why have",
                       u"will’ve": u"will have", u"won’t": u"will not", u"won’t’ve": u"will not have",
                       u"would’ve": u"would have", u"wouldn’t": u"would not", u"wouldn’t’ve": u"would not have",
                       u"y’all": u"you all", u"y’all’d": u"you all would",u"y’all’d’ve": u"you all would have",
                       u"y’all’re": u"you all are",u"y’all’ve": u"you all have",u"you’d": u"you would",
                       u"you’d’ve": u"you would have", u"you’ll": u"you will", u"you’ll’ve": u"you will have",
                       u"you’re": u"you are", u"you’ve": u"you have", u"cat’s": u"cat is",u" whatapp ": u" whatapps ",
                       u" whatssapp ": u" whatapps ",u" whatssap ": u" whatapps ",u" whatspp ": u" whatapps ",
                       u" whastapp ": u" whatapps ",u" whatsap ": u" whatapps ",u" whassap ": u" whatapps ",
                       u" watapps ": u" whatapps ",u"wetfood": u"wet food",u"intetested": u"interested",
                       u"领养条件，": u"领养条件",u"谢谢。": u"谢谢",
                       u"别打我，记住，我有反抗的牙齿，但我不会咬你。remember": u"别打我，记住，我有反抗的牙齿，但我不会咬你。",
                       u"有你。do": u"有你。",u"名字name": u"名字",u"year，": u"year",u"work，your": u"work your",
                       u"too，will": u"too will",u"timtams": u"timtam",u"spay。": u"spay",u"shoulder，a": u"shoulder a",
                       u"sherpherd": u"shepherd",u"sherphed": u"shepherd",u"sherperd": u"shepherd",
                       u"sherpard": u"shepherd",u"serious。": u"serious",u"remember，i": u"remember i",
                       u"recover，": u"recover",u"refundable指定期限内结扎后会全数奉还": u"refundable",
                       u"puchong区，有没有人有增添家庭成员？": u"puchong",u"puchong救的": u"puchong",
                       u"puchong，": u"puchong",u"month。": u"month",u"month，": u"month",
                       u"microchip（做狗牌一定要有主人的电话号码）": u"microchip",u"maju。": u"maju",u"maincoone": u"maincoon",
                       u"lumpur。": u"lumpur",u"location：阿里玛，大山脚": u"location",u"life🐾🐾": u"life",
                       u"kibble，": u"kibble",u"home…": u"home",u"hand，but": u"hand but",u"hair，a": u"hair a",
                       u"grey、brown": u"grey brown",u"gray，": u"gray",u"free免费": u"free",u"food，or": u"food or",
                       u"dog／dog": u"dog",u"dijumpa": u"dijumpai",u"dibela": u"dibelai",
                       u"beauuuuuuuuutiful": u"beautiful",u"adopt🙏": u"adopt",u"addopt": u"adopt",
                      u"enxiety": u"anxiety", u"vaksin": u"vaccine"}

train = pd.read_csv('../../input/petfinder-adoption-prediction/train/train.csv')
test = pd.read_csv('../../input/petfinder-adoption-prediction/test/test.csv')
train = pd.concat([train, test], sort=True)
train[['Description', 'Name']] = train[['Description', 'Name']].astype(str)
train["Description"] = [analyzer_embed(text) for text in train["Description"]]

path = "../../input/pymagnitude-data/glove.840B.300d.magnitude"
result, oov = w2v_pymagnitude(train["Description"], path, name="glove")
result.to_feather("../feature/glove_mag.feather")

1235005 9232 0.992580191715887


In [ ]:
1218578 8140 0.9933644081198776

In [10]:
import collections

c = collections.Counter(oov)
[(l,k) for k,l in sorted([(j,i) for i,j in c.items()], reverse=True)]

[('\uf0e8', 175),
 ('whatapps', 118),
 ('deflea', 60),
 ('😊', 41),
 ('谢谢您', 33),
 ('花', 32),
 ('分钟的时间', 32),
 ('vaksin', 30),
 ('tempoh', 29),
 ('ibunya', 28),
 ('tuntutan', 27),
 ('photostated', 27),
 ('enxiety', 27),
 ('️', 26),
 ('makhota', 25),
 ('membela', 24),
 ('dibuang', 24),
 ('selamatkan', 23),
 ('prodiet', 23),
 ('🐶', 22),
 ('perlukan', 22),
 ('memerlukan', 22),
 ('kurap', 22),
 ('deticked', 22),
 ('感恩', 21),
 ('也希望你多花', 21),
 ('terbiar', 21),
 ('dijumpaii', 21),
 ('的资料', 20),
 ('或者想在带回家前看看这个可爱的小家伙', 20),
 ('分钟看看', 20),
 ('tompok', 20),
 ('kembirikan', 20),
 ('考虑收养', 19),
 ('menyusu', 19),
 ('msger', 18),
 ('欢迎联络', 17),
 ('如果您真的有兴趣要带', 17),
 ('如果您愿意给', 17),
 ('再安排地点与时间', 17),
 ('一个温暖的家', 17),
 ('penyayang', 17),
 ('lincah', 17),
 ('kelaparan', 17),
 ('dijaga', 17),
 ('回家', 16),
 ('lepaskan', 16),
 ('diselamatkan', 16),
 ('给个人资料', 15),
 ('病了一定要看兽医', 15),
 ('每个月需告诉我们狗狗的情况', 15),
 ('条规', 15),
 ('家人同意', 15),
 ('学生不能领养', 15),
 ('地点', 15),
 ('像儿女一样对待', 15),
 ('menjaganya', 15),
 (

In [22]:
import collections

c = collections.Counter(oov)
[(l,k) for k,l in sorted([(j,i) for i,j in c.items()], reverse=True)]

[('\uf0e8', 175),
 ('perai', 121),
 ('dbkl', 121),
 ('seberang', 116),
 ('i’m', 87),
 ('seekor', 84),
 ('don’t', 77),
 ('haiwan', 67),
 ('deflea', 60),
 ('whatapps', 59),
 ('mahsuri', 56),
 ('mddb', 43),
 ('kurungan', 43),
 ('mpsj', 42),
 ('😊', 41),
 ('intetested', 41),
 ('wetfood', 40),
 ('whatapp', 37),
 ('kemuning', 37),
 ('can’t', 37),
 ('perbandaran', 34),
 ('kuchai', 33),
 ('she’s', 31),
 ('perindustrian', 31),
 ('jantina', 31),
 ('vaksin', 30),
 ('he’s', 30),
 ('tempoh', 29),
 ('lebuh', 29),
 ('seksyen', 28),
 ('kucing2', 28),
 ('kinrara', 28),
 ('ibunya', 28),
 ('tuntutan', 27),
 ('photostated', 27),
 ('pelabuhan', 27),
 ('it’s', 27),
 ('enxiety', 27),
 ('rm80', 26),
 ('babycat', 26),
 ('makhota', 25),
 ('cat’s', 25),
 ('sekor', 24),
 ('membela', 24),
 ('dibuang', 24),
 ('selamatkan', 23),
 ('prodiet', 23),
 ('mertajam', 23),
 ('i’ll', 23),
 ('perlukan', 22),
 ('memerlukan', 22),
 ('mbpj', 22),
 ('kurap', 22),
 ('deticked', 22),
 ('🐶', 21),
 ('terbiar', 21),
 ('dijumpai', 21),


In [14]:
#original
1190305 | 20832 | 0.9827996337326

#analyze
904043 | 30216 | 0.9676577908267407

#kaeru
923315 | 11086 | 0.9881357147520176

1215616 11102 0.9909498352514596

#小文字化
5086174 | 28019 | 0.9945213252608965

SyntaxError: invalid syntax (<ipython-input-14-baf6b8e89df4>, line 10)

In [20]:
[(l,k) for k,l in sorted([(j,i) for i,j in c.items()], reverse=True)]

[('\uf0e8', 175),
 ('perai', 121),
 ('dbkl', 121),
 ('seberang', 116),
 ('i’m', 87),
 ('seekor', 84),
 ('don’t', 77),
 ('haiwan', 67),
 ('deflea', 60),
 ('whatapps', 59),
 ('mahsuri', 56),
 ('mddb', 43),
 ('kurungan', 43),
 ('mpsj', 42),
 ('😊', 41),
 ('intetested', 41),
 ('wetfood', 40),
 ('whatapp', 37),
 ('kemuning', 37),
 ('can’t', 37),
 ('perbandaran', 34),
 ('kuchai', 33),
 ('she’s', 31),
 ('perindustrian', 31),
 ('jantina', 31),
 ('vaksin', 30),
 ('he’s', 30),
 ('tempoh', 29),
 ('lebuh', 29),
 ('seksyen', 28),
 ('kucing2', 28),
 ('kinrara', 28),
 ('ibunya', 28),
 ('tuntutan', 27),
 ('photostated', 27),
 ('pelabuhan', 27),
 ('it’s', 27),
 ('enxiety', 27),
 ('rm80', 26),
 ('babycat', 26),
 ('makhota', 25),
 ('cat’s', 25),
 ('sekor', 24),
 ('membela', 24),
 ('dibuang', 24),
 ('selamatkan', 23),
 ('prodiet', 23),
 ('mertajam', 23),
 ('i’ll', 23),
 ('perlukan', 22),
 ('memerlukan', 22),
 ('mbpj', 22),
 ('kurap', 22),
 ('deticked', 22),
 ('🐶', 21),
 ('terbiar', 21),
 ('dijumpai', 21),
